# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [2]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
# Verificando o 'tipo_renda' dos valores missing para a variável 'tempo_emprego'
df[df['tempo_emprego'].isna() == True]['tipo_renda'].value_counts()

tipo_renda
Pensionista    2573
Name: count, dtype: int64

In [5]:
# Preenchendo com '0' os valores missing pois os mesmos são pensionistas
df['tempo_emprego'].fillna(0, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          15000 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [6]:
# Excluindo as variáveis que não serão utilizadas na Patsy
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace = True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [7]:
# Verificando o item de maior frequência para a variável 'qt_pessoas_residencia'
df['qt_pessoas_residencia'].value_counts()

qt_pessoas_residencia
2.0     8181
1.0     2752
3.0     2551
4.0     1311
5.0      179
6.0       18
9.0        5
15.0       2
7.0        1
Name: count, dtype: int64

In [8]:
# Verificando o item de maior frequência para a variável 'qtd_filhos'
df['qtd_filhos'].value_counts()

qtd_filhos
0     10376
1      3037
2      1376
3       185
4        17
7         5
14        2
5         2
Name: count, dtype: int64

In [9]:
# Criando as variáveis dummies para as variáveis qualitativas
df_dummies = patsy.dmatrix('sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + C(qt_pessoas_residencia, Treatment(2))', df, return_type = 'dataframe')
df_dummies.head()

,Intercept,sexo[T.M],posse_de_veiculo[T.True],posse_de_imovel[T.True],C(qtd_filhos)[T.1],C(qtd_filhos)[T.2],C(qtd_filhos)[T.3],C(qtd_filhos)[T.4],C(qtd_filhos)[T.5],C(qtd_filhos)[T.7],...,"C(qt_pessoas_residencia, Treatment(2))[T.1.0]","C(qt_pessoas_residencia, Treatment(2))[T.3.0]","C(qt_pessoas_residencia, Treatment(2))[T.4.0]","C(qt_pessoas_residencia, Treatment(2))[T.5.0]","C(qt_pessoas_residencia, Treatment(2))[T.6.0]","C(qt_pessoas_residencia, Treatment(2))[T.7.0]","C(qt_pessoas_residencia, Treatment(2))[T.9.0]","C(qt_pessoas_residencia, Treatment(2))[T.15.0]",idade,tempo_emprego
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,6.602740
1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,7.183562
2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,0.838356
3,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,4.846575
4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,4.293151


In [10]:
# Mantendo a categoria mais frequente como casela de referência
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + C(qt_pessoas_residencia, Treatment(2))', data = df)
x

DesignMatrix with shape (15000, 38)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'C(qtd_filhos)[T.1]',
     'C(qtd_filhos)[T.2]',
     'C(qtd_filhos)[T.3]',
     'C(qtd_filhos)[T.4]',
     'C(qtd_filhos)[T.5]',
     'C(qtd_filhos)[T.7]',
     'C(qtd_filhos)[T.14]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'estado_civil[T.Separado]',
     'estado_civil[T.Solteiro]',
     'estado_civil[T.União]',
     'estado_civil[T.Viúvo]',
     'tipo_residencia[T.Casa]',
     'tipo_residencia[T.Com os pais]',
     'tipo_residencia[T.Comunitário]',
     'tipo_residencia[T.Estúdio]',
     'tipo_residencia[T.Governamental]',
     'C(qt_pessoas_residencia, Treatment(2))[T.1.0]',
     'C(qt_pesso

In [11]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     235.8
Date:                Sat, 14 Oct 2023   Prob (F-statistic):               0.00
Time:                        12:19:22   Log-Likelihood:                -16207.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14965   BIC:                         3.275e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          7.1995      0.082     87.363      0.000       7.038       7.361
sexo[T.M]                                          0.7947      0.014     57.614      0.000       0.768       0.822
posse_de_veiculo[T.True]                           0.0344      0.013      2.638      0.008       0.009       0.060
posse_de_imovel[T.True]                            0.0904      0.013      7.026      0.000       0.065       0.116
C(qtd_filhos)[T.1]                                -0.2115      0.108     -1.955      0.051      -0.424       0.001
C(qtd_filhos)[T.2]                                -0.5081      0.216     -2.353      0.019      -0.931      -0.085
C(qtd_filhos)[T.3]                                -0.4248      0.430     -0.989      0.323      -1.267       0.417
C(qtd_filhos)[T.4]                                 0.1858      0.424      0.438      0.661      -0.645       1.016
C(qtd_filhos)[T.5]                                 0.1616      0.406      0.398      0.690      -0.634       0.957
C(qtd_filhos)[T.7]                                 0.0226      0.160      0.141      0.887      -0.291       0.336
C(qtd_filhos)[T.14]                               -0.4385      0.258     -1.699      0.089      -0.945       0.068
tipo_renda[T.Bolsista]                             0.2094      0.238      0.878      0.380      -0.258       0.677
tipo_renda[T.Empresário]                           0.1539      0.015     10.427      0.000       0.125       0.183
tipo_renda[T.Pensionista]                          0.2793      0.024     11.744      0.000       0.233       0.326
tipo_renda[T.Servidor público]                     0.0552      0.022      2.515      0.012       0.012       0.098
educacao[T.Pós graduação]                          0.0493      0.151      0.326      0.745      -0.247       0.346
educacao[T.Secundário]                            -0.0715      0.056     -1.270      0.204      -0.182       0.039
educacao[T.Superior completo]                      0.0470      0.057      0.829      0.407      -0.064       0.158
educacao[T.Superior incompleto]                   -0.1005      0.063     -1.583      0.113      -0.225       0.024
estado_civil[T.Separado]                           0.2655      0.108      2.454      0.014       0.053       0.477
estado_civil[T.Solteiro]                           0.2176      0.106      2.045      0.041       0.009       0.426
estado_civil[T.União]                             -0.0376      0.023     -1.635      0.102      -0.083       0.007
estado_civil[T.Viúvo]                              0.2717      0.110      2.473      0.013       0.056       0.487
tipo_residencia[T.Casa]                           -0.0315      0.052     -0.606      0.545   

**Avaliando os parâmetros:**
* O modelo explica pouco mais de **um terço dos eventos**, conforme $R^2$;
* A variável **'qt_filhos'** a partir de **3** é pouca significante para o modelo;
* O mesmo pode ser dito para a variável **'qt_pessoas_residencia'** a partir de **5**;
* As variáveis **'educacao'** e **'tipo_residencia'** também apresentam pouca significância; e
* Ter **pós-graduação** apresenta pouco significância para o modelo diferente do que eu pensava.

In [12]:
# Retirando a variável menos significante: 'tipo_residência
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + educacao + estado_civil + idade + tempo_emprego + C(qt_pessoas_residencia, Treatment(2))', data = df)
x

DesignMatrix with shape (15000, 33)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'C(qtd_filhos)[T.1]',
     'C(qtd_filhos)[T.2]',
     'C(qtd_filhos)[T.3]',
     'C(qtd_filhos)[T.4]',
     'C(qtd_filhos)[T.5]',
     'C(qtd_filhos)[T.7]',
     'C(qtd_filhos)[T.14]',
     'tipo_renda[T.Bolsista]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'estado_civil[T.Separado]',
     'estado_civil[T.Solteiro]',
     'estado_civil[T.União]',
     'estado_civil[T.Viúvo]',
     'C(qt_pessoas_residencia, Treatment(2))[T.1.0]',
     'C(qt_pessoas_residencia, Treatment(2))[T.3.0]',
     'C(qt_pessoas_residencia, Treatment(2))[T.4.0]',
     'C(qt_pessoas_residencia, Treatment(2))[T.5.0]',
     'C(qt_pessoas_residencia, Treatment

In [13]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.349
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     276.4
Date:                Sat, 14 Oct 2023   Prob (F-statistic):               0.00
Time:                        12:19:26   Log-Likelihood:                -16209.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14970   BIC:                         3.271e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                          7.1719      0.065    109.792      0.000       7.044       7.300
sexo[T.M]                                          0.7959      0.014     57.859      0.000       0.769       0.823
posse_de_veiculo[T.True]                           0.0343      0.013      2.628      0.009       0.009       0.060
posse_de_imovel[T.True]                            0.0876      0.013      6.926      0.000       0.063       0.112
C(qtd_filhos)[T.1]                                -0.2118      0.108     -1.958      0.050      -0.424       0.000
C(qtd_filhos)[T.2]                                -0.5105      0.216     -2.364      0.018      -0.934      -0.087
C(qtd_filhos)[T.3]                                -0.4263      0.430     -0.992      0.321      -1.268       0.416
C(qtd_filhos)[T.4]                                 0.1643      0.423      0.388      0.698      -0.665       0.994
C(qtd_filhos)[T.5]                                 0.1401      0.405      0.346      0.729      -0.654       0.934
C(qtd_filhos)[T.7]                                 0.0205      0.160      0.128      0.898      -0.293       0.334
C(qtd_filhos)[T.14]                               -0.4403      0.258     -1.706      0.088      -0.946       0.066
tipo_renda[T.Bolsista]                             0.2095      0.238      0.879      0.380      -0.258       0.677
tipo_renda[T.Empresário]                           0.1542      0.015     10.468      0.000       0.125       0.183
tipo_renda[T.Pensionista]                          0.2802      0.024     11.795      0.000       0.234       0.327
tipo_renda[T.Servidor público]                     0.0561      0.022      2.556      0.011       0.013       0.099
educacao[T.Pós graduação]                          0.0490      0.151      0.324      0.746      -0.248       0.345
educacao[T.Secundário]                            -0.0692      0.056     -1.232      0.218      -0.179       0.041
educacao[T.Superior completo]                      0.0486      0.057      0.857      0.391      -0.063       0.160
educacao[T.Superior incompleto]                   -0.0987      0.063     -1.555      0.120      -0.223       0.026
estado_civil[T.Separado]                           0.2672      0.108      2.471      0.013       0.055       0.479
estado_civil[T.Solteiro]                           0.2203      0.106      2.071      0.038       0.012       0.429
estado_civil[T.União]                             -0.0375      0.023     -1.630      0.103      -0.083       0.008
estado_civil[T.Viúvo]                              0.2729      0.110      2.484      0.013       0.058       0.488
C(qt_pessoas_residencia, Treatment(2))[T.1.0]     -0.2483      0.106     -2.332      0.020   

**Avaliando os parâmetros:**
* Não houve mudança no $R^2$ e nem no ajustado;
* Há em geral uma **pequena melhora** para os **P-Value** abaixo de **0.05**; e
* Uma **pequena elevação** para os **P-Value** pouco significantes.

In [14]:
# Removendo as variáveis menos significantes
df_ajustado = df[(df.qtd_filhos < 3)&(df.qt_pessoas_residencia < 5)&(df.tipo_renda != 'Bolsista')&(df.estado_civil != 'União')]
df_ajustado = df_ajustado.drop(['educacao', 'tipo_residencia'], axis = 1)
df_ajustado.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,estado_civil,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Solteiro,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Casado,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Casado,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Casado,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Solteiro,33,4.293151,1.0,6475.97


In [15]:
# Gerando os valores de 'x' e 'y' após a retiradas das variáveis
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + C(qtd_filhos) + tipo_renda + estado_civil + idade + tempo_emprego + C(qt_pessoas_residencia, Treatment(2))', data = df_ajustado)
x

DesignMatrix with shape (13711, 17)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_veiculo[T.True]',
     'posse_de_imovel[T.True]',
     'C(qtd_filhos)[T.1]',
     'C(qtd_filhos)[T.2]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Pensionista]',
     'tipo_renda[T.Servidor público]',
     'estado_civil[T.Separado]',
     'estado_civil[T.Solteiro]',
     'estado_civil[T.Viúvo]',
     'C(qt_pessoas_residencia, Treatment(2))[T.1.0]',
     'C(qt_pessoas_residencia, Treatment(2))[T.3.0]',
     'C(qt_pessoas_residencia, Treatment(2))[T.4.0]',
     'idade',
     'tempo_emprego']
  Terms:
    'Intercept' (column 0)
    'sexo' (column 1)
    'posse_de_veiculo' (column 2)
    'posse_de_imovel' (column 3)
    'C(qtd_filhos)' (columns 4:6)
    'tipo_renda' (columns 6:9)
    'estado_civil' (columns 9:12)
    'C(qt_pessoas_residencia, Treatment(2))' (columns 12:15)
    'idade' (column 15)
    'tempo_emprego' (column 16)
  (to view full data, use np.asarray(this_obj))

In [16]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.352
Method:                 Least Squares   F-statistic:                     466.5
Date:                Sat, 14 Oct 2023   Prob (F-statistic):               0.00
Time:                        12:19:31   Log-Likelihood:                -14826.
No. Observations:               13711   AIC:                         2.969e+04
Df Residuals:                   13694   BIC:                         2.981e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         7.1472      0.036    197.341      0.000       7.076       7.218
sexo[T.M]                                         0.8068      0.014     56.044      0.000       0.779       0.835
posse_de_veiculo[T.True]                          0.0457      0.014      3.366      0.001       0.019       0.072
posse_de_imovel[T.True]                           0.0902      0.013      6.813      0.000       0.064       0.116
C(qtd_filhos)[T.1]                               -0.2138      0.113     -1.889      0.059      -0.436       0.008
C(qtd_filhos)[T.2]                               -0.5136      0.225     -2.288      0.022      -0.954      -0.074
tipo_renda[T.Empresário]                          0.1669      0.015     10.809      0.000       0.137       0.197
tipo_renda[T.Pensionista]                         0.2872      0.024     11.736      0.000       0.239       0.335
tipo_renda[T.Servidor público]                    0.0772      0.023      3.394      0.001       0.033       0.122
estado_civil[T.Separado]                          0.2820      0.113      2.496      0.013       0.061       0.503
estado_civil[T.Solteiro]                          0.2309      0.111      2.076      0.038       0.013       0.449
estado_civil[T.Viúvo]                             0.2769      0.115      2.415      0.016       0.052       0.502
C(qt_pessoas_residencia, Treatment(2))[T.1.0]    -0.2561      0.111     -2.300      0.021      -0.474      -0.038
C(qt_pessoas_residencia, Treatment(2))[T.3.0]     0.2378      0.114      2.094      0.036       0.015       0.460
C(qt_pessoas_residencia, Treatment(2))[T.4.0]     0.5991      0.225      2.659      0.008       0.157       1.041
idade                                             0.0050      0.001      6.405      0.000       0.003       0.007
tempo_emprego                                     0.0618      0.001     58.271      0.000       0.060       0.064
==============================================================================
Omnibus:                        2.225   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.329   Jarque-Bera (JB):                2.234
Skew:                           0.014   Prob(JB):                        0.327
Kurtosis:                       3.056   Cond. No.                     3.09e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Avaliando os parâmetros:**
* O modelo revisado apresenta uma pequena melhora no $R^2$ normal e no ajustado;
* A variável **'qt_filhos'** com valor igual a **1** ficou ligeramente acima de 0.05 para o **P-Value**; e
* As demais ficaram abaixo de 0.05.